# Complete Explanation: Shor's QFT Addition Implementation

## 1. Mathematical Foundation

### Quantum Fourier Transform Basis
The QFT transforms computational basis states into a superposition where arithmetic becomes phase manipulation:

$$QFT|x\rangle = \frac{1}{\sqrt{2^n}} \sum_{k=0}^{2^n-1} e^{i\frac{2\pi xk}{2^n}} |k\rangle$$

In this basis, **addition becomes phase multiplication**:
- To add constant $a$ to $|x\rangle$, we need: $|\phi(x + a)\rangle$
- This requires applying phase $e^{i\frac{2\pi ak}{2^n}}$ to each $|k\rangle$ component

### Key Insight: Distributed Phase Encoding
Instead of applying one big phase to each $|k\rangle$, we can **distribute** the phase across individual qubits using the binary representation of $k$.

For $k = k_0 + k_1 \cdot 2 + k_2 \cdot 4 + \ldots$ (little-endian), we can write:
$$e^{i\frac{2\pi ak}{2^n}} = e^{i\frac{2\pi a(k_0 + k_1 \cdot 2 + k_2 \cdot 4 + \ldots)}{2^n}}$$

$$= e^{i\frac{2\pi ak_0}{2^n}} \cdot e^{i\frac{2\pi ak_1 \cdot 2}{2^n}} \cdot e^{i\frac{2\pi ak_2 \cdot 4}{2^n}} \cdots$$

## 2. The Chain Implementation

### Step 1: `_get_angles(a, n)` - Calculate Individual Qubit Phases

```python
def _get_angles(a, n):
    bits_little_endian = (bin(int(a))[2:].zfill(n))[::-1]
    angles = np.zeros(n)
    
    for i in range(n):
        for j in range(i + 1):
            k = i - j
            if bits_little_endian[j] == "1":
                angles[i] += pow(2, -k)
    
    return angles * np.pi
```

**Mathematical Derivation:**
For qubit $i$, we need the phase that will be applied when that qubit is $|1\rangle$.

From the QFT structure, qubit $i$ in Fourier space represents:
$$|\phi_i\rangle = \frac{1}{\sqrt{2}}(|0\rangle + e^{i\phi_i}|1\rangle)$$

The phase $\phi_i$ must encode the contribution of bit $i$ to the addition of $a$.

**The Algorithm:**
- For each qubit position $i$
- Look at all lower-order bits $j \leq i$ of $a$
- If bit $j$ is set, add $2^{-(i-j)}$ to the angle
- Multiply by $\pi$ for the final phase

**Why this works:**
This calculates exactly the phase needed so that when applied to the QFT representation, it performs modular addition by $a$.

### Step 2: `_phi_add_gate(angles)` - Create Quantum Circuit

```python
def _phi_add_gate(angles):
    circuit = QuantumCircuit(len(angles), name="phi_add_a")
    for i, angle in enumerate(angles):
        circuit.p(angle, i)  # Phase gate P(angle) on qubit i
    return circuit.to_gate()
```

**What this does:**
- Creates a quantum circuit with $n$ qubits
- Applies phase gate $P(\text{angle}[i])$ to qubit $i$
- Phase gate: $P(\theta) = \begin{pmatrix} 1 & 0 \\ 0 & e^{i\theta} \end{pmatrix}$

**Effect:**
- When qubit $i$ is $|0\rangle$: no change
- When qubit $i$ is $|1\rangle$: multiply by $e^{i \cdot \text{angle}[i]}$

### Step 3: Integration in Modular Arithmetic

In `_controlled_multiple_mod_N`, the phi_add_gate is used as:

```python
def append_adder(adder, constant, idx):
    partial_constant = (pow(2, idx, N) * constant) % N
    angles = self._get_angles(partial_constant, n + 1)
    # Apply the angles...
```

**The Scaling:**
- `pow(2, idx, N)` computes $2^{\text{idx}} \bmod N$
- This implements **binary multiplication**: $\text{constant} \times 2^{\text{idx}}$
- The modular arithmetic ensures results stay within valid range

## 3. Complete Example: Adding 5 to |2⟩ in 3-bit System

### Mathematical Setup
- $n = 3$ qubits, so $N = 2^3 = 8$
- Want to compute: $|2\rangle + 5 = |7\rangle$ (mod 8)
- Binary: $a = 5 = 101_2$ (little-endian: "101")

### Step-by-Step Calculation

**Step 1: Calculate angles for adding 5**

For $a = 5$, `bits_little_endian = "101"`:

**Qubit 0 ($i = 0$):**
- $j = 0$: $k = 0 - 0 = 0$, bit[0] = "1" → add $2^{-0} = 1$
- Final: $\text{angles}[0] = 1 \times \pi = \pi$

**Qubit 1 ($i = 1$):**
- $j = 0$: $k = 1 - 0 = 1$, bit[0] = "1" → add $2^{-1} = 0.5$
- $j = 1$: $k = 1 - 1 = 0$, bit[1] = "0" → add nothing
- Final: $\text{angles}[1] = 0.5 \times \pi = 0.5\pi$

**Qubit 2 ($i = 2$):**
- $j = 0$: $k = 2 - 0 = 2$, bit[0] = "1" → add $2^{-2} = 0.25$
- $j = 1$: $k = 2 - 1 = 1$, bit[1] = "0" → add nothing
- $j = 2$: $k = 2 - 2 = 0$, bit[2] = "1" → add $2^{-0} = 1$
- Final: $\text{angles}[2] = (0.25 + 1) \times \pi = 1.25\pi$

**Result:** $\text{angles} = [\pi, 0.5\pi, 1.25\pi]$

### Step 2: Apply to QFT state

Starting with $|2\rangle = |010\rangle$, after QFT:
$$|\phi(2)\rangle = \frac{1}{2\sqrt{2}} \sum_{k=0}^{7} e^{i\frac{2\pi \cdot 2 \cdot k}{8}} |k\rangle$$

Apply phase gates:
- Qubit 0: $P(\pi)$ → multiply by $e^{i\pi} = -1$ when qubit 0 is $|1\rangle$
- Qubit 1: $P(0.5\pi)$ → multiply by $e^{i0.5\pi} = i$ when qubit 1 is $|1\rangle$  
- Qubit 2: $P(1.25\pi)$ → multiply by $e^{i1.25\pi} = -\frac{1}{\sqrt{2}}(1+i)$ when qubit 2 is $|1\rangle$

### Step 3: The magic happens

Through the mathematical structure of QFT, these individual qubit phases **combine constructively** to:
1. **Cancel out** the phases for all states except $|7\rangle$
2. **Amplify** the amplitude for $|7\rangle$ to 1
3. **Destructively interfere** for all other states

After inverse QFT: $|\phi(2+5)\rangle \rightarrow |7\rangle$ with probability 1.

## 4. Why This Method Works

### Distributed Phase Computation
Instead of computing one large phase per state, Shor's method:
1. **Decomposes** the required phase into qubit-specific contributions
2. **Applies** these phases independently to each qubit
3. **Relies** on QFT structure to recombine them correctly

### Modular Arithmetic Integration
The method naturally handles:
- **Overflow**: Modular arithmetic keeps results in valid range
- **Scaling**: Powers of 2 for binary multiplication
- **Controlled operations**: Easy to make conditional on other qubits

### Efficiency Benefits
- **Circuit depth**: Only $n$ phase gates instead of complex classical addition
- **Parallelization**: All phase gates can be applied simultaneously
- **Reusability**: Same angles work for any input state

## 5. Connection to Shor's Algorithm

In Shor's factoring algorithm, this QFT addition is used for:
1. **Modular exponentiation**: $a^x \bmod N$
2. **Controlled multiplication**: Based on bits of the exponent
3. **Period finding**: The core of the factoring algorithm

The `_get_angles` method enables efficient quantum circuits for these operations, making Shor's algorithm practical for quantum computers.

## 6. Advanced Building Blocks: Controlled Operations

### `_double_controlled_phi_add_mod_N` - Conditional Modular Addition

```python
def _double_controlled_phi_add_mod_N(self, angles, c_phi_add_N, iphi_add_N, qft, iqft):
    ctrl_qreg = QuantumRegister(2, "ctrl")      # 2 control qubits
    b_qreg = QuantumRegister(len(angles), "b")  # Target register (in QFT basis)
    flag_qreg = QuantumRegister(1, "flag")      # Auxiliary flag qubit
    
    circuit = QuantumCircuit(ctrl_qreg, b_qreg, flag_qreg, name="ccphi_add_a_mod_N")
    
    cc_phi_add_a = self._phi_add_gate(angles).control(2)  # Double-controlled addition
    cc_iphi_add_a = cc_phi_add_a.inverse()
    
    # The actual circuit operations...
    circuit.append(cc_phi_add_a, [*ctrl_qreg, *b_qreg])
    circuit.append(iphi_add_N, b_qreg)
    circuit.append(iqft, b_qreg)
    circuit.cx(b_qreg[-1], flag_qreg[0])
    circuit.append(qft, b_qreg)
    circuit.append(c_phi_add_N, [*flag_qreg, *b_qreg])
    circuit.append(cc_iphi_add_a, [*ctrl_qreg, *b_qreg])
    circuit.append(iqft, b_qreg)
    circuit.x(b_qreg[-1])
    circuit.cx(b_qreg[-1], flag_qreg[0])
    circuit.x(b_qreg[-1])
    circuit.append(qft, b_qreg)
    circuit.append(cc_phi_add_a, [*ctrl_qreg, *b_qreg])
    
    return circuit
```

**Purpose:** Performs modular addition $b \leftarrow (b + a) \bmod N$ **only when both control qubits are |1⟩**.

**Mathematical Foundation:**
- **Problem**: Simple addition might overflow: $b + a \geq N$
- **Solution**: Use "addition-subtraction-correction" method
- **Key insight**: Modular arithmetic requires checking and correcting overflow

**How the circuit works:**

1. **Step 1**: `cc_phi_add_a` - Add $a$ to $b$ (controlled by both control qubits)
   - Result: $b' = b + a$ (might be $\geq N$)

2. **Step 2**: `iphi_add_N` - Subtract $N$ from result
   - Result: $b'' = b + a - N$ (might be negative)

3. **Step 3**: Check if result is negative using the flag qubit
   - `iqft` → `cx(b_qreg[-1], flag_qreg[0])` → `qft`
   - The most significant bit indicates if $b + a - N < 0$

4. **Step 4**: If negative, add $N$ back (controlled by flag)
   - `c_phi_add_N` controlled by flag qubit
   - If flag is |1⟩ (negative result), add $N$ back

5. **Step 5**: Undo the original addition if it wasn't needed
   - Complex sequence to ensure proper modular arithmetic

**Example:**
- $N = 7$, $b = 5$, $a = 3$
- $b + a = 8 \geq 7$, so subtract 7: $8 - 7 = 1$
- Result: $(5 + 3) \bmod 7 = 1$ ✓

### `_controlled_multiple_mod_N` - Modular Multiplication

```python
def _controlled_multiple_mod_N(self, n, N, a, c_phi_add_N, iphi_add_N, qft, iqft):
    ctrl_qreg = QuantumRegister(1, "ctrl")       # 1 control qubit
    x_qreg = QuantumRegister(n, "x")             # Input number x
    b_qreg = QuantumRegister(n + 1, "b")         # Accumulator (starts at 0)
    flag_qreg = QuantumRegister(1, "flag")       # Auxiliary flag
    
    def append_adder(adder, constant, idx):
        partial_constant = (pow(2, idx, N) * constant) % N
        angles = self._get_angles(partial_constant, n + 1)
        bound = adder.assign_parameters({angle_params: angles})
        circuit.append(bound, [*ctrl_qreg, x_qreg[idx], *b_qreg, *flag_qreg])
    
    circuit.append(qft, b_qreg)  # Transform to QFT basis
    
    for i in range(n):
        append_adder(modulo_adder, a, i)  # Add a×2^i if x[i] = 1
    
    circuit.append(iqft, b_qreg)  # Transform back to computational basis
    
    # Swap x and b registers
    for i in range(n):
        circuit.cswap(ctrl_qreg, x_qreg[i], b_qreg[i])
    
    # Undo the additions (for reversibility)
    circuit.append(qft, b_qreg)
    a_inv = pow(a, -1, mod=N)  # Modular inverse of a
    for i in reversed(range(n)):
        append_adder(modulo_adder_inv, a_inv, i)
    circuit.append(iqft, b_qreg)
    
    return circuit.to_instruction()
```

**Purpose:** Computes $x \leftarrow (a \times x) \bmod N$ when control qubit is |1⟩.

**Mathematical Foundation:**
Uses **binary multiplication algorithm**:
$a \times x = a \times (x_0 \cdot 2^0 + x_1 \cdot 2^1 + x_2 \cdot 2^2 + \ldots)$
$= a \times x_0 \cdot 2^0 + a \times x_1 \cdot 2^1 + a \times x_2 \cdot 2^2 + \ldots$

**Algorithm Steps:**

1. **Initialize**: Accumulator $b = 0$

2. **For each bit $i$ of $x$**:
   - If $x[i] = 1$: Add $(a \times 2^i) \bmod N$ to accumulator
   - Use `append_adder` with `partial_constant = (a × 2^i) mod N`

3. **Swap registers**: $x \leftrightarrow b$ (now $b$ contains $a \times x \bmod N$)

4. **Cleanup**: Undo additions using modular inverse for reversibility

**Example with $a = 3$, $x = 5$, $N = 7$:**
- $x = 5 = 101_2$ (bits: $x_0=1, x_1=0, x_2=1$)
- Step 1: $x_0=1$, add $(3 \times 2^0) \bmod 7 = 3$, accumulator = 3
- Step 2: $x_1=0$, skip
- Step 3: $x_2=1$, add $(3 \times 2^2) \bmod 7 = 12 \bmod 7 = 5$, accumulator = $(3+5) \bmod 7 = 1$
- Result: $(3 \times 5) \bmod 7 = 15 \bmod 7 = 1$ ✓

### `append_adder` Function - The Scaling Mechanism

```python
def append_adder(adder, constant, idx):
    partial_constant = (pow(2, idx, N) * constant) % N
    angles = self._get_angles(partial_constant, n + 1)
    bound = adder.assign_parameters({angle_params: angles})
    circuit.append(bound, [*ctrl_qreg, x_qreg[idx], *b_qreg, *flag_qreg])
```

**Purpose:** Adds $(constant \times 2^{idx}) \bmod N$ to the accumulator, controlled by bit $idx$ of $x$.

**Key Components:**

1. **Scaling calculation**: `pow(2, idx, N)`
   - Computes $2^{idx} \bmod N$ efficiently
   - Handles large powers modulo $N$

2. **Partial constant**: `(pow(2, idx, N) * constant) % N`
   - The actual value to add: $(constant \times 2^{idx}) \bmod N$

3. **Angle calculation**: `self._get_angles(partial_constant, n + 1)`
   - Uses our familiar `_get_angles` method
   - Calculates phases needed for this specific addition

4. **Parameter binding**: `adder.assign_parameters({angle_params: angles})`
   - Takes the generic `modulo_adder` circuit template
   - Fills in the specific angles for this addition

5. **Controlled execution**: Controlled by `x_qreg[idx]`
   - Addition happens **only if** bit $idx$ of $x$ is |1⟩

**Example with $constant = 3$, $idx = 2$, $N = 7$:**
- `pow(2, 2, 7) = 4`
- `partial_constant = (4 × 3) % 7 = 12 % 7 = 5`
- `angles = _get_angles(5, 4)` = angles for adding 5
- Addition of 5 happens only if `x[2] = 1`

## 7. How Everything Connects

### The Complete Multiplication Circuit Flow:

1. **Input**: Want to compute $(a \times x) \bmod N$

2. **Binary decomposition**: $x = \sum_{i=0}^{n-1} x_i \cdot 2^i$

3. **For each bit $i$ where $x_i = 1$**:
   - Calculate: $\text{contribution} = (a \times 2^i) \bmod N$
   - Use `append_adder` to add this contribution
   - `append_adder` calls `_get_angles` to get the right phases
   - `_get_angles` computes phases for adding that specific contribution

4. **Accumulation**: All contributions sum up to $(a \times x) \bmod N$

5. **Cleanup**: Undo operations for reversibility using modular inverse

### Example: Computing $(3 \times 13) \bmod 7$ with 4-bit numbers

**Setup:** $a = 3$, $x = 13 = 1101_2$, $N = 7$

**Bit-by-bit calculation:**
- Bit 0: $x_0 = 1$, add $(3 \times 2^0) \bmod 7 = 3$
- Bit 1: $x_1 = 0$, skip
- Bit 2: $x_2 = 1$, add $(3 \times 2^2) \bmod 7 = 12 \bmod 7 = 5$ 
- Bit 3: $x_3 = 1$, add $(3 \times 2^3) \bmod 7 = 24 \bmod 7 = 3$

**Total:** $(3 + 5 + 3) \bmod 7 = 11 \bmod 7 = 4$

**Verification:** $(3 \times 13) \bmod 7 = 39 \bmod 7 = 4$ ✓

## 8. The Crown Jewel: `_power_mod_N` - Modular Exponentiation

```python
def _power_mod_N(self, n: int, N: int, a: int) -> Instruction:
    """Implements modular exponentiation a^x as an instruction."""
    up_qreg = QuantumRegister(2 * n, name="up")      # Control register (x in superposition)
    down_qreg = QuantumRegister(n, name="down")       # Target register (starts as |1⟩)
    aux_qreg = QuantumRegister(n + 2, name="aux")     # Auxiliary qubits
    
    circuit = QuantumCircuit(up_qreg, down_qreg, aux_qreg, name=f"power_mod_N_{a}_{N}")
    
    qft = QFT(n + 1, do_swaps=False).to_gate()
    iqft = qft.inverse()
    
    phi_add_N = self._phi_add_gate(self._get_angles(N, n + 1))
    iphi_add_N = phi_add_N.inverse()
    c_phi_add_N = phi_add_N.control(1)
    
    for i in range(2 * n):
        partial_a = pow(a, pow(2, i), N)  # Compute a^(2^i) mod N
        modulo_multiplier = self._controlled_multiple_mod_N(
            n, N, partial_a, c_phi_add_N, iphi_add_N, qft, iqft
        )
        circuit.append(modulo_multiplier, [up_qreg[i], *down_qreg, *aux_qreg])
    
    return circuit.to_instruction()
```

**Purpose:** This is the **heart of Shor's algorithm** - it computes $a^x \bmod N$ where $x$ is in quantum superposition, enabling the quantum period-finding that makes factoring efficient.

### Mathematical Foundation: Binary Exponentiation

**The Core Insight:** Any integer $x$ can be written in binary as:
$x = x_0 \cdot 2^0 + x_1 \cdot 2^1 + x_2 \cdot 2^2 + \ldots + x_{2n-1} \cdot 2^{2n-1}$

Therefore:
$a^x = a^{x_0 \cdot 2^0 + x_1 \cdot 2^1 + x_2 \cdot 2^2 + \ldots}$

$= a^{x_0 \cdot 2^0} \cdot a^{x_1 \cdot 2^1} \cdot a^{x_2 \cdot 2^2} \cdot \ldots$

$= (a^{2^0})^{x_0} \cdot (a^{2^1})^{x_1} \cdot (a^{2^2})^{x_2} \cdot \ldots$

**Key insight:** Each factor $(a^{2^i})^{x_i}$ equals either:
- $1$ if $x_i = 0$ (don't multiply)
- $a^{2^i} \bmod N$ if $x_i = 1$ (do multiply)

### Algorithm Breakdown

#### **Step 1: Precompute Powers**
```python
partial_a = pow(a, pow(2, i), N)  # Compute a^(2^i) mod N
```

For each bit position $i$, precompute $a^{2^i} \bmod N$:
- $a^{2^0} = a^1 \bmod N$
- $a^{2^1} = a^2 \bmod N$ 
- $a^{2^2} = a^4 \bmod N$
- $a^{2^3} = a^8 \bmod N$
- ... and so on

**Example with $a = 3$, $N = 7$:**
- $3^{2^0} = 3^1 = 3 \bmod 7 = 3$
- $3^{2^1} = 3^2 = 9 \bmod 7 = 2$ 
- $3^{2^2} = 3^4 = 81 \bmod 7 = 4$
- $3^{2^3} = 3^8 = 6561 \bmod 7 = 1$

#### **Step 2: Controlled Multiplications**
```python
modulo_multiplier = self._controlled_multiple_mod_N(
    n, N, partial_a, c_phi_add_N, iphi_add_N, qft, iqft
)
circuit.append(modulo_multiplier, [up_qreg[i], *down_qreg, *aux_qreg])
```

For each bit $i$ of the exponent $x$:
- **If** `up_qreg[i]` (bit $i$ of $x$) is $|1\rangle$: multiply by $a^{2^i} \bmod N$
- **If** `up_qreg[i]` is $|0\rangle$: do nothing (multiply by 1)

This uses the `_controlled_multiple_mod_N` we explained earlier.

### Complete Example: Computing $3^{13} \bmod 7$

**Setup:** $a = 3$, $x = 13 = 1101_2$, $N = 7$

#### **Step 1: Precomputed Powers**
- $3^{2^0} = 3^1 = 3$
- $3^{2^1} = 3^2 = 2$ 
- $3^{2^2} = 3^4 = 4$
- $3^{2^3} = 3^8 = 1$

#### **Step 2: Binary Decomposition of $x = 13$**
$13 = 1101_2$ means:
- Bit 0: $x_0 = 1$ → multiply by $3^{2^0} = 3$
- Bit 1: $x_1 = 0$ → skip (multiply by 1)
- Bit 2: $x_2 = 1$ → multiply by $3^{2^2} = 4$  
- Bit 3: $x_3 = 1$ → multiply by $3^{2^3} = 1$

#### **Step 3: Quantum Circuit Execution**
Starting with target register = $|1\rangle$:

1. **Bit 0 controlled multiplication:** $1 \times 3 = 3$
2. **Bit 1:** Skip (control is $|0\rangle$)
3. **Bit 2 controlled multiplication:** $3 \times 4 = 12 \bmod 7 = 5$
4. **Bit 3 controlled multiplication:** $5 \times 1 = 5$

**Result:** $3^{13} \bmod 7 = 5$

**Verification:** $3^{13} = 1594323 \bmod 7 = 5$ ✓

### Register Layout and Quantum Superposition

#### **Register Structure:**
- **`up_qreg` (2n qubits)**: Contains $x$ in **quantum superposition**
- **`down_qreg` (n qubits)**: Target register, starts as $|1\rangle$, becomes $|a^x \bmod N\rangle$
- **`aux_qreg` (n+2 qubits)**: Auxiliary qubits for intermediate calculations

#### **The Quantum Magic:**
When `up_qreg` contains a **superposition** of all possible values of $x$:
$|x\rangle = \frac{1}{\sqrt{2^{2n}}} \sum_{x=0}^{2^{2n}-1} |x\rangle$

The circuit creates:
$\frac{1}{\sqrt{2^{2n}}} \sum_{x=0}^{2^{2n}-1} |x\rangle |a^x \bmod N\rangle$

This **simultaneously computes** $a^x \bmod N$ for **all possible values of $x$**!

### Integration with Shor's Algorithm

#### **Phase Estimation Context:**
In Shor's algorithm, this circuit is used within quantum phase estimation to find the period $r$ of the function $f(x) = a^x \bmod N$.

**The period $r$** is the smallest positive integer such that:
$a^r \equiv 1 \pmod{N}$

#### **How Period Finding Works:**
1. **Prepare superposition:** $\frac{1}{\sqrt{2^{2n}}} \sum_{x=0}^{2^{2n}-1} |x\rangle |1\rangle$

2. **Apply modular exponentiation:** $\frac{1}{\sqrt{2^{2n}}} \sum_{x=0}^{2^{2n}-1} |x\rangle |a^x \bmod N\rangle$

3. **Quantum Fourier Transform:** Extracts the period $r$ from the quantum state

4. **Classical post-processing:** Uses $r$ to factor $N$

### Why This Circuit is Crucial

#### **Exponential Speedup:**
- **Classical approach:** Computing $a^x \bmod N$ for one value of $x$ takes $O(\log x)$ multiplications
- **Quantum approach:** This circuit computes $a^x \bmod N$ for **all $2^{2n}$ values** simultaneously!

#### **Efficient Implementation:**
- **Only $2n$ controlled multiplications** (one per bit of $x$)
- **Each multiplication** uses the building blocks we've explained
- **Total complexity:** Polynomial in $n = \log N$

#### **Reversible and Unitary:**
- **All operations are reversible** (required for quantum computing)
- **Preserves quantum superposition** throughout the computation
- **Enables quantum interference** needed for period extraction

### The Complete Chain Connection

This `_power_mod_N` method represents the **culmination** of our entire building block chain:

1. **`_get_angles`** → calculates basic phase rotations
2. **`_phi_add_gate`** → implements QFT addition  
3. **`_double_controlled_phi_add_mod_N`** → safe modular addition
4. **`_controlled_multiple_mod_N`** → modular multiplication using binary decomposition
5. **`_power_mod_N`** → modular exponentiation using repeated squaring

Each level builds on the previous ones, ultimately enabling the **quantum modular exponentiation** that makes Shor's factoring algorithm exponentially faster than any known classical algorithm.

## 9. The Complete Shor's Circuit: `construct_circuit`

```python
def construct_circuit(self, N: int, a: int = 2, measurement: bool = False) -> QuantumCircuit:
    """Construct quantum part of the algorithm."""
    self._validate_input(N, a)
    
    n = N.bit_length()
    
    up_qreg = QuantumRegister(2 * n, name="up")      # Control register for exponent
    down_qreg = QuantumRegister(n, name="down")       # Target register 
    aux_qreg = QuantumRegister(n + 2, name="aux")     # Auxiliary qubits
    
    circuit = QuantumCircuit(up_qreg, down_qreg, aux_qreg, name=f"Shor(N={N}, a={a})")
    
    # Step 1: Create uniform superposition of all possible exponents
    circuit.h(up_qreg)
    
    # Step 2: Initialize target register to |1⟩
    circuit.x(down_qreg[0])
    
    # Step 3: Apply modular exponentiation
    modulo_power = self._power_mod_N(n, N, a)
    circuit.append(modulo_power, circuit.qubits)
    
    # Step 4: Apply inverse QFT for period extraction
    iqft = QFT(len(up_qreg)).inverse().to_gate()
    circuit.append(iqft, up_qreg)
    
    # Step 5: Optional measurement
    if measurement:
        up_cqreg = ClassicalRegister(2 * n, name="m")
        circuit.add_register(up_cqreg)
        circuit.measure(up_qreg, up_cqreg)
    
    return circuit
```

**Purpose:** This method constructs the **complete quantum circuit for Shor's factoring algorithm** - the circuit that finds the period of $a^x \bmod N$, which can then be used to factor $N$.

### Mathematical Foundation: Quantum Phase Estimation

Shor's algorithm is fundamentally a **quantum phase estimation** problem. We want to find the period $r$ of the function:
$f(x) = a^x \bmod N$

The period $r$ is the smallest positive integer such that:
$a^r \equiv 1 \pmod{N}$

Once we know $r$, we can factor $N$ using the mathematical relationship:
$\gcd(a^{r/2} \pm 1, N)$
often gives non-trivial factors of $N$.

### Step-by-Step Circuit Analysis

#### **Step 1: Quantum Register Setup**
```python
n = N.bit_length()
up_qreg = QuantumRegister(2 * n, name="up")      # 2n qubits
down_qreg = QuantumRegister(n, name="down")       # n qubits  
aux_qreg = QuantumRegister(n + 2, name="aux")     # n+2 qubits
```

**Register sizing logic:**
- **`n = N.bit_length()`**: Number of bits needed to represent $N$
- **`up_qreg` (2n qubits)**: Control register for the exponent $x$
  - Needs $2n$ qubits for sufficient precision in period finding
  - Will contain superposition of exponents from $0$ to $2^{2n}-1$
- **`down_qreg` (n qubits)**: Target register that will hold $a^x \bmod N$
- **`aux_qreg` (n+2 qubits)**: Auxiliary qubits needed for modular arithmetic

#### **Step 2: Create Uniform Superposition**
```python
circuit.h(up_qreg)  # Apply Hadamard to all 2n qubits
```

**Mathematical effect:**
$|0\rangle^{\otimes 2n} \xrightarrow{H^{\otimes 2n}} \frac{1}{\sqrt{2^{2n}}} \sum_{x=0}^{2^{2n}-1} |x\rangle$

This creates an **equal superposition** of all possible exponent values from $0$ to $2^{2n}-1$.

**Why this matters:**
- We don't know what the period $r$ is
- By putting the exponent in superposition, we can **probe all possible periods simultaneously**
- This is the source of the quantum speedup

#### **Step 3: Initialize Target Register**
```python
circuit.x(down_qreg[0])  # Set down_qreg to |1⟩
```

**Mathematical effect:**
$|0\rangle^{\otimes n} \xrightarrow{X_0} |1\rangle = |00...01\rangle$

**Why start with |1⟩:**
- We want to compute $a^x \bmod N$ starting from 1
- $a^0 = 1$, so starting with |1⟩ gives the correct base case
- The modular exponentiation will multiply this initial state by appropriate powers

#### **Step 4: Apply Modular Exponentiation**
```python
modulo_power = self._power_mod_N(n, N, a)
circuit.append(modulo_power, circuit.qubits)
```

**Mathematical effect:**
$\frac{1}{\sqrt{2^{2n}}} \sum_{x=0}^{2^{2n}-1} |x\rangle |1\rangle |0\rangle^{\otimes (n+2)} \xrightarrow{\text{modulo\_power}}$
$\frac{1}{\sqrt{2^{2n}}} \sum_{x=0}^{2^{2n}-1} |x\rangle |a^x \bmod N\rangle |\text{aux}\rangle$

**Key insight:**
- This step **entangles** the exponent register with the result register
- Now the quantum state contains **all possible values** of $a^x \bmod N$ simultaneously
- The auxiliary register helps with intermediate calculations but doesn't affect the final result

#### **Step 5: Quantum Period Extraction via Inverse QFT**
```python
iqft = QFT(len(up_qreg)).inverse().to_gate()
circuit.append(iqft, up_qreg)
```

**Mathematical effect:**
This is the **heart of the period-finding algorithm**. The inverse QFT transforms the uniform superposition into a state that reveals the period.

**How it works mathematically:**
1. **Before IQFT**: The state has the form $\sum_{x} |x\rangle |a^x \bmod N\rangle$

2. **Key observation**: Due to periodicity, $a^x \bmod N = a^{x+r} \bmod N = a^{x+2r} \bmod N = ...$

3. **After IQFT**: The quantum interference causes **constructive interference** for multiples of $\frac{2^{2n}}{r}$ and **destructive interference** elsewhere

4. **Result**: High probability of measuring values close to $k \cdot \frac{2^{2n}}{r}$ for integer $k$

### Complete Example: Finding the Period of $3^x \bmod 7$

**Setup:** $N = 7$, $a = 3$, so $n = 7.\text{bit\_length}() = 3$

#### **Step 1: Register sizes**
- `up_qreg`: $2 \times 3 = 6$ qubits (can represent exponents 0-63)
- `down_qreg`: $3$ qubits (can represent values 0-7)
- `aux_qreg`: $3 + 2 = 5$ qubits

#### **Step 2: After superposition creation**
$\frac{1}{\sqrt{64}} \sum_{x=0}^{63} |x\rangle |1\rangle |0\rangle^{\otimes 5}$

#### **Step 3: After modular exponentiation**
$\frac{1}{\sqrt{64}} \sum_{x=0}^{63} |x\rangle |3^x \bmod 7\rangle |\text{aux}\rangle$

**The periodic pattern:**
- $3^0 \bmod 7 = 1$
- $3^1 \bmod 7 = 3$ 
- $3^2 \bmod 7 = 2$
- $3^3 \bmod 7 = 6$
- $3^4 \bmod 7 = 4$
- $3^5 \bmod 7 = 5$
- $3^6 \bmod 7 = 1$ ← **Period r = 6**
- $3^7 \bmod 7 = 3$ (repeats)

#### **Step 4: After inverse QFT**
The IQFT causes quantum interference that makes measurements likely to give values near:
- $k \cdot \frac{64}{6} = k \cdot 10.67$ for $k = 0, 1, 2, 3, 4, 5$

So we're likely to measure: $0, 11, 21, 32, 43, 53$ (approximately)

#### **Step 5: Classical post-processing**
From a measurement result like 21, we can extract the period:
- $\frac{21}{64} \approx \frac{2}{6} = \frac{1}{3}$
- Using continued fractions: period $r = 6$

### Why This Circuit Achieves Quantum Speedup

#### **Classical Approach:**
- **Brute force**: Try all possible periods up to $N-1$
- **Time complexity**: $O(N)$ which is exponential in the number of bits
- **For large $N$**: Completely impractical

#### **Quantum Approach:**
- **Superposition**: Test all periods simultaneously
- **Time complexity**: Polynomial in $\log N$ (the number of bits)
- **For large $N$**: Exponentially faster than classical

### Connection to Factoring

Once we find the period $r$, factoring works as follows:

1. **Check if $r$ is even**: If odd, try a different value of $a$

2. **Compute**: $\gcd(a^{r/2} - 1, N)$ and $\gcd(a^{r/2} + 1, N)$

3. **Example with $N = 15$, $a = 2$, period $r = 4$:**
   - $2^{4/2} - 1 = 2^2 - 1 = 3$
   - $2^{4/2} + 1 = 2^2 + 1 = 5$ 
   - $\gcd(3, 15) = 3$ and $\gcd(5, 15) = 5$
   - **Factors found**: $15 = 3 \times 5$ ✓

### The Complete Building Block Chain in Action

This final circuit showcases the **entire chain** we've built:

1. **`_get_angles`**: Provides the mathematical foundation for QFT arithmetic
2. **`_phi_add_gate`**: Implements basic addition in Fourier space
3. **`_double_controlled_phi_add_mod_N`**: Enables safe modular addition
4. **`_controlled_multiple_mod_N`**: Implements modular multiplication  
5. **`_power_mod_N`**: Performs modular exponentiation using repeated squaring
6. **`construct_circuit`**: Combines everything into Shor's period-finding algorithm

Each level builds on the previous ones, ultimately enabling the **quantum factoring** that threatens current cryptographic systems.

### Measurement and Classical Post-Processing

```python
if measurement:
    up_cqreg = ClassicalRegister(2 * n, name="m")
    circuit.add_register(up_cqreg)
    circuit.measure(up_qreg, up_cqreg)
```

**What happens after measurement:**
1. **Get measurement result**: A classical bit string representing some value $m$
2. **Extract fraction**: $\frac{m}{2^{2n}}$ approximates $\frac{s}{r}$ for some integers $s, r$
3. **Use continued fractions**: To find the period $r$ from this fraction
4. **Verify period**: Check that $a^r \equiv 1 \pmod{N}$
5. **Factor**: Use the period to find factors of $N$

This completes Shor's algorithm - from the simple `_get_angles` calculation all the way to quantum factoring of large integers!

## Summary

The entire chain works by:
1. **Mathematical insight**: Addition in QFT space = phase manipulation
2. **Clever decomposition**: Break complex phases into simple qubit rotations  
3. **Efficient implementation**: Use binary representation for systematic calculation
4. **Quantum interference**: Let QFT structure handle the complex recombination
5. **Period finding**: Use quantum phase estimation to extract periodicities
6. **Classical post-processing**: Convert quantum measurement to actual factors

This is why Shor's `get_angles` method is so elegant - it transforms a complex quantum arithmetic operation into a series of simple phase rotations that quantum computers can execute efficiently, ultimately enabling the factoring of integers that would be impossible to factor classically!